ID3算法

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('watermelon_dataset3.csv',index_col=0)

1.预处理


In [2]:
df.loc[df['密度']>df['密度'].median(),'密度']='高'
df.loc[df['密度']!='高','密度']='低'
df.loc[df['含糖率']>df['含糖率'].median(),'含糖率']='高'
df.loc[df['含糖率']!='高','含糖率']='低'

label=df['好瓜']=='是'
df=df.drop(['好瓜'],axis=1)
df


,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率
编号,,,,,,,,
1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,高,高
2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,高,高
3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,高,高
4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,高,高
5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,低,高
6,青绿,稍蜷,浊响,清晰,稍凹,软粘,低,高
7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,低,低
8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,低,低
9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,高,低


输出Label标签

In [3]:
label


编号
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
Name: 好瓜, dtype: bool

2.计算信息熵


In [4]:
from math import log

def entropy(data,label):
    ent = 0
    for l in label.unique():
        p = sum(label==l)/len(label)
        ent-= p*log(p,2)
    return ent


3.建树与训练

信息增益：

$Gain(D,a) = Ent(D) - \sum_{v=1}^V\dfrac{|D^v|}{|D|}H(D^v)$

- 创建树结构
    - 1.初始化创建节点类来进行操作
    - 2.划分条件
        - 2.1 当前结点包含的样本全属于同一类别，无需划分;
        - 2.2 当前属性集为空或者所有样本在所有属性上取值相同,无法划分;
        - 2.3 当前结点包含的样本集合为空,不能划分。


In [5]:
class node:
    def __init__(self,df,label):
        self.data=df
        self.label=label

        self.criterion=None
        self.children=[]
        self.ent = entropy(df,label)
        self.leaf=-1
        self.type=''

    def divide(self):
        if len(self.label.unique())==1:
            self.leaf=list(self.label)[0]
            return
        if len(self.data.columns)==0:
            self.leaf=self.label.value_counts().index[0]
            return

        #寻找最优属性
        IGs={}
        for c in self.data.columns:
            IG=self.ent
            for typ in self.data[c].unique():
                ent = entropy(self.data.loc[self.data[c]==typ],self.label[self.data[c]==typ])
                IG -= ent*sum(self.data[c]==typ)/len(self.data[c])
            IGs[IG]=c

        if max(IGs.keys())>0:
            self.criterion=IGs[max(IGs.keys())]
            c=self.criterion
            for typ in self.data[c].unique():
                #建立子节点
                new_data = self.data.loc[self.data[c]==typ].drop(c,axis=1)
                new_label = self.label[self.data[c]==typ]
                new = node(new_data,new_label)
                new.type=str(typ)
                new.divide()
                self.children.append(new)
        else:
            self.leaf=self.label.value_counts().index[0]
            return


In [6]:
TREE = node(df,label)
TREE.divide()


4.结果展示

- 1 代表还有子节点
True和False表示是不是好瓜



In [7]:
def bfs(node):
    if node.criterion:
        print("按%s划分"%node.criterion)
        for c in node.children:
            print(c.type,end='\t')
        print()
        for c in node.children:
            print(c.leaf,end='\t')
        print()
        for c in node.children:
            if c.leaf==-1:print('如果%s%s-->'%((node.criterion,c.type)),end='')
            bfs(c)

In [8]:
bfs(TREE)


按纹理划分
清晰	稍糊	模糊	
-1	-1	False	
如果纹理清晰-->按触感划分
硬滑	软粘	
True	-1	
如果触感软粘-->按脐部划分
稍凹	平坦	
-1	False	
如果脐部稍凹-->按色泽划分
青绿	乌黑	
True	False	
如果纹理稍糊-->按密度划分
低	高	
True	False	
